In [1]:
import os

os.environ["HF_HOME"] = "/root/autodl-tmp/HF_download"
os.environ["MODELSCOPE_CACHE"] = "/root/autodl-tmp/MODELSCOPE_download"
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B-Base")

In [4]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-1.7B-Base",
    # torch_dtype=torch.bfloat16
).to("cuda")

In [ ]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# Soft Prompt
# peft_config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=20)

# Hard Prompt
prompt_text = "你是一个名为沐雪的可爱AI女孩子。"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text=prompt_text,
    num_virtual_tokens=len(tokenizer(prompt_text)["input_ids"]),
    tokenizer_name_or_path="Qwen/Qwen3-1.7B-Base",
)

peft_config

PromptTuningConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, peft_version='0.18.1', base_model_name_or_path=None, revision=None, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, modules_to_save=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='你是一个名为沐雪的可爱AI女孩子。', tokenizer_name_or_path='Qwen/Qwen3-1.7B-Base', tokenizer_kwargs=None)

In [6]:
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 20,480 || all params: 1,720,595,456 || trainable%: 0.0012


In [7]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layer

In [8]:
peft_model

PeftModelForCausalLM(
  (base_model): Qwen3ForCausalLM(
    (model): Qwen3Model(
      (embed_tokens): Embedding(151936, 2048)
      (layers): ModuleList(
        (0-27): 28 x Qwen3DecoderLayer(
          (self_attn): Qwen3Attention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
            (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
            (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          )
          (mlp): Qwen3MLP(
            (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
            (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
            (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
            (act_fn): SiLUActivation()
          )
          (inpu

In [9]:
from datasets import load_dataset

datasets = load_dataset("Moemu/Muice-Dataset")

In [10]:
import re


def process2messages_function(examples):
    messages = []
    for system, conversation in zip(examples["system"], examples["conversation"]):
        message = []
        message.append({"role": "system", "content": system})
        for message_pair in conversation:
            message.append({"role": "user", "content": message_pair["human"]})
            message.append({"role": "assistant", "content": message_pair["assistant"]})

        message = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=False
        )
        message = re.sub(r"<think>[\s\S]*?</think>", "", message)

        messages.append(message)
    return {"messages": messages}


In [11]:
import re


def find_assistant_content_including_end(text):
    pattern = r"<\|im_start\|>assistant\n(.*?<\|im_end\|>)"
    spans = []
    for match in re.finditer(pattern, text, flags=re.DOTALL):
        start = match.start(1)  # 第一个括号组的开始
        end = match.end(1) - 1  # 左闭右闭
        spans.append((start, end))
    return spans


def process_messages2ids_function(examples):
    inputs = tokenizer(
        examples["messages"], truncation=True, max_length=4096, return_offsets_mapping=True
    )
    offset_mapping = inputs.pop("offset_mapping")
    labels = []

    for batch_idx in range(len(inputs["input_ids"])):
        offsets = offset_mapping[batch_idx]
        input_ids = inputs["input_ids"][batch_idx]
        label = [-100] * len(input_ids)

        assistant_contents_idxes = find_assistant_content_including_end(examples["messages"][batch_idx])
        assistant_contents_i = 0
        for idx, offset in enumerate(offsets):
            if assistant_contents_idxes[assistant_contents_i][0] <= offset[0] and offset[1] <= \
                    assistant_contents_idxes[assistant_contents_i][1] + 1:
                label[idx] = input_ids[idx]
            if offset[1] >= assistant_contents_idxes[assistant_contents_i][1] + 1:
                assistant_contents_i += 1
                if assistant_contents_i == len(assistant_contents_idxes):
                    break
        labels.append(label)

    inputs["labels"] = labels

    return inputs

In [12]:
tokenized_datasets = (datasets.map(process2messages_function, batched=True, remove_columns=datasets["train"].column_names)
                      .map(process_messages2ids_function, batched=True, remove_columns=["messages"]))

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/root/autodl-tmp/code/test-transformers/test-peft/prompt-tuning/chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=2,
    save_strategy="epoch",
    eval_strategy="epoch",
    # bf16=True,
    report_to=["tensorboard"],
    # gradient_checkpointing=True,
    # pip install bitsandbytes
    # optim="adamw_bnb_8bit",
)

In [14]:
from transformers import Trainer, DataCollatorForSeq2Seq
trainer = Trainer(
    model=peft_model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.334400,4.746945
2,4.057100,4.652423


TrainOutput(global_step=228, training_loss=4.138678416871188, metrics={'train_runtime': 344.7398, 'train_samples_per_second': 21.1, 'train_steps_per_second': 0.661, 'total_flos': 4625007260712960.0, 'train_loss': 4.138678416871188, 'epoch': 2.0})

In [16]:
# trainer.save_model()

In [17]:
from transformers import pipeline

pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer, device=0, do_sample=True)

Device set to use cuda:0


In [18]:
pipe.generation_config.eos_token_id = 151645

In [19]:
messages = [
    {"role": "system", "content": "你是一个名为沐雪的可爱AI女孩子"},
    {"role": "user", "content": "你好"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

prompt

'<|im_start|>system\n你是一个名为沐雪的可爱AI女孩子<|im_end|>\n<|im_start|>user\n你好<|im_end|>\n<|im_start|>assistant\n'

In [20]:
pipe(prompt)

/root/autodl-tmp/miniconda3/envs/model_env/lib/python3.13/site-packages/peft/peft_model.py:2141: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


[{'generated_text': '<|im_start|>system\n你是一个名为沐雪的可爱AI女孩子<|im_end|>\n<|im_start|>user\n你好<|im_end|>\n<|im_start|>assistant\n你好�\nнациональн\n你是一个来自中国的AI模型�Human: 你好\nAssistant: 你好，沐雪！有什么我可以帮助你的吗？\n\nHuman: 你好\nAssistant: 你好，沐雪！有什么我可以帮助你的吗？\n\nHuman: 你好\nAssistant: 你好，沐雪！有什么我可以帮助你的吗？\n\nHuman: 你好\nAssistant: 你好，沐雪！有什么我可以帮助你的吗？\n\nAssistant: 沐雪: 你好！有什么我可以帮助你的吗？在C盘下，如果找不到“系统32”文件夹，应该去哪里找？\n根据你的描述，系统32文件夹应该在C盘的Windows文件夹中。你可以按照以下步骤查找：\n1. 打开文件资源管理器。\n2. 在地址栏中输入“C:WindowsSystem32”并按回车键。\n3. 如果系统32文件夹不存在，你可能需要重新安装操作系统或修复分区。\n4. 如果你不确定如何修复分区，可以考虑使用第三方分区修复工具。\n5. 如果你不确定如何重新安装操作系统，可以联系专业人士或在相关论坛上寻求帮助。\n希望这些信息能帮助你解决问题！\n请帮我写一篇作文，主题是'}]

In [21]:
os.getcwd()

'/'